## Information
This script contains: 
1. One self-contained block that solves equation 18 of the 1976/1981 paper.
3. One self-contained block that solves equation 20 of the 1981 paper.
4. One self-contained block that compares them 

## Packages and specificities for plot

In [ ]:
#Import relevant packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import erfc
from scipy.integrate import odeint
from scipy.integrate import quadrature
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
import time

#Use Latex and serif font
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## 1. Solution of Equation 18, Prosperetti 1976/1981
This code generates amplitude as a function of time by solving the IODE, equation 18, from Prosperetti's 1976 papaer, Viscous effects on small-amplitude surface waves.

In [ ]:
##Functions

#1.Defines an ODE function without integral
def model0_0(a,t):
    dadt = a[1]
    d2adt2 = -16 * (np.pi**2 / np.sqrt(Ga)) * a[1] - 2 * np.pi * a[0]
    return [dadt,d2adt2]

#2.Defines the ODE with integral
def model(t,a,guess):
    #The time variable that the model receives could be messing up the integral term 
    interpolator = interp1d(guess[:,0], guess[:,1], kind='linear') 
    integral_term, _ = quadrature(lambda m: integrate_term(t,m) * interpolator(m), 0, t, tol=1e-11, rtol=1e-8, maxiter=2000)
    dadt = a[1]
    d2adt2 = -16 * (np.pi**2 / np.sqrt(Ga)) * a[1] - 2 * np.pi * a[0] + 64 * (np.pi**4 / Ga) * integral_term
    return [dadt, d2adt2]

#3. Calculates the f(s) part of the integrand f(s)a(s)
def integrate_term(t, m):
    if (t==0) :    
        c_1 = 1e-20
    else:
        c_1 = (4 * (np.pi**3 / np.sqrt(Ga)) * (t - m))**(-1/2)
    c_2 = np.exp(-4 * (np.pi**2 / np.sqrt(Ga)) * (t - m))
    c_3 = erfc(4 * (np.pi**2 / np.sqrt(Ga)) * (t - m))**(1/2)
    return c_1 * c_2 - c_3

##Solves the IODE

#Solves Simple ODE 
Ga = 1e5
t = np.linspace(0, 26, 5000)
a0 = [0.01, -1e-4]
sol0_0 = odeint(model0_0, a0, t)

#Plots Simple ODE a(t) as a check
plt.plot(t, sol0_0[:,0])
plt.axhline(0, color='black', linewidth=0.2)  # Add horizontal line at y=0
plt.axvline(0, color='black', linewidth=0.2)  # Add vertical line at x=0
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('ODE Solution with Axis Lines')
plt.grid(True)  #Add grid lines
plt.show()


#Parameters for solving the ODE 
counter = 1 
err = 1e4 
Tol = np.array([1e-8,1e-8])
t_span = (t[0], t[-1])
guess_data = np.column_stack((t, sol0_0))


# Start measuring the execution time
start_time = time.time()

#Solves the IODE
while (err > Tol).all():
    sol = solve_ivp(model, t_span, a0, args=(guess_data,), t_eval=t, method='RK45')
    err = np.sum((sol.y-guess_data[:, 1:].T)**2, axis=1)
    formatted_err = [f'{e:8.2e}' for e in err]
    print(f'{counter:4d}: {" ".join(formatted_err)}')
    wt = 0.5
    guess_data = np.column_stack((sol.t, (1-wt)*guess_data[:, 1:] + wt*sol.y.T))
    counter = counter+1

    
#Plots a(t) for the IODE
fig, ax = plt.subplots(1, 1, figsize=(10,5))
ax.plot(t, sol.y[0], 'k-', lw=2)
ax.set_xlabel(r'$\sqrt{gt^2/\lambda_w}$', fontsize=20)
ax.set_ylabel(r'$\zeta/\lambda_w$', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlim([0., 4.0])
#ax.set_ylim([0., max(guess[:,0])])
ax.legend()
plt.show()

# Results for execution time
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

#Places the data from IODE into variables
t1 = t
A1 = sol.y[0]

## 2.Equation 20, Prosperetti 1981

This code generates (t2,A2) using Equation 20 and 21 from Prosperetti's 1981 paper, Motion of two superposed viscous fluids, to get the evolution of the interface as a function of time. 

In [ ]:
##Functions:
#1.calculate Zis using cyclical permutation of zis 
def cycl_roots(z):
    Z = np.zeros(0)
    for z_i in z:
        mult = 1
        for z_j in z:
            if z_i != z_j:
                mult = mult*(z_j-z_i)
        Z = np.append(Z,mult)
    return Z


#2.Calculates Analytical expression for the Amplitude
def ampl(t,a0,u0,Ga,Beta,z,Z):
    b = 1-4*Beta
    Amp_p1 = a0 * (erfc(k2v*t)**0.5) * 32 * b * np.pi**3 / (64 * b * np.pi**3 + Ga)
    Amp_p2 = 0
    for i in range(4):
        Amp_p21 = (z[i]/Z[i]) * (2*np.pi*A_0/(z[i]**2-k2v)-U_0) 
        Amp_p22 = np.exp((z[i]**2-k2v)*t) * erfc(z[i]*t**0.5)
        Amp_p2 = Amp_p2 + (Amp_p21 * Amp_p22)
    return Amp_p1 + Amp_p2 


#Initializing variables 
p_u = 1
p_l = 0
Ga = 1e5 
A_0 = 0.01
U_0 = -1e-4 #Taken from the estimation of a'(0)


#Calculating zi, Zi, non dimensional Amplitude
B = (p_u * p_l)/((p_u + p_l)**2)
k2v = 4 * (np.pi**2/np.sqrt(Ga))
p = np.poly1d([1, -4*B*k2v**0.5, 2*(1-6*B)*k2v, 4*(1-3*B)*k2v**1.5,(1-4*B)*k2v**2+(2*np.pi)])
z = p.roots 
Z = cycl_roots(z)

t2 = np.linspace(0, 26, 5000)
A2 = ampl(t2,A_0,U_0,Ga,B,z,Z).real

### 3. Comparing Equation 18 and 20
This code generates a plot with data from Equation 18 from 1976 and 20 from 1981, in order to compare the two equations. For them to be identical, either pu = 0 or pl = 0. You also need Prosperetti.dat, which is in the github repository Prosperetti.

In [ ]:
#Import data from Prosperetti-gravity.h
df = pd.read_csv('Prosperetti.dat', sep=' ')
t = df.iloc[:, 1].to_numpy()
Amp = df.iloc[:, 3].to_numpy()

#Set plot
fig, ax = plt.subplots(1, 1, figsize=(10,5))

#Data
ax.plot(t1, abs(A1), 'r-', lw=2, label='Prosperetti_18_1976')
ax.plot(t2, abs(A2), 'k-', lw=2, label='Prosperetti_20_1981')

#Specificities
ax.set_xlabel(r'$\sqrt{gt^2/\lambda_w}$', fontsize=15)
ax.set_ylabel(r'$\zeta/\lambda_w$', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.set_xlim([0., 27])
#ax.set_ylim([0., A0])
ax.legend()
plt.title("Comparison between Eq. 18 and 20")
#plt.savefig("Comparison_pa_Ga=150004")
plt.show()